# Парсинг HTML

In [1]:
# Установим сразу зависимости, необходимые для примера
%pip install lxml bs4 ipython

Note: you may need to restart the kernel to use updated packages.


## Страница для примера

In [2]:

html_data = """
<html>
<body>
    <div id="note">
        <p class="abc">Какая-то умная надпись</p>
        <p> Номер телефона 8(906)123-45-32 </p>
    </div>
    <table id="my_cool_table">
        <thead>
            <tr abc="def">
                <td>#</td>
                <td>Имя</td>
                <td>Фамилия</td>
                <td>Возраст</td>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td class="id">1</td>
                <td class="name">Иван</td>
                <td class="surname">Петров</td>
                <td class="age">27</td>
            </tr>
            <tr>
                <td class="id">2</td>
                <td class="name">Петр</td>
                <td class="surname">Иванов</td>
                <td class="age">31</td>
            </tr>
            <tr>
                <td class="id">3</td>
                <td class="name">Петр</td>
                <td class="surname">Сидоров</td>
                <td class="age">19</td>
            </tr>
        </tbody>
    </table>
</body>
</html>

"""

In [3]:
from IPython.display import display, HTML
display(HTML(html_data))


#,Имя,Фамилия,Возраст
1,Иван,Петров,27
2,Петр,Иванов,31
3,Петр,Сидоров,19


## Способы достать информацию из HTML
1. Без учета структры. Используем регулярные выражения.
2. С учетом древовидной структуры. Обращаемся к узлам дерева при помощи селекторов.

<b>HTML - это дерево.</b>
<img src="./html_tree.svg">

### Селекторы

#### XPath
XPath - XML Path Language. Подходит для работы с любыми XML документами, не только HTML.

<b>Синтаксис</b>

Можно посмотреть, например, в https://bugbug.io/blog/testing-frameworks/the-ultimate-xpath-cheat-sheet/ или в https://msiter.ru/tutorials/xpath/syntax.
* / - задает абсолютный путь (надо указать полный путь в дереве от корня до отыскиваемого узла);
* // - позволяет отыскать узлы во всем документе.
* . - текущий узел;
* .. - родительский узел;
* element - выбирает все узлы с именем element;
* * - любой узел;
* @attribute - позволяет получить значение атрибута attribute;
* text() - позволяет получить текст, который находится внутри узла;
* [predicate] - позволяет получить узлы, которые удовлетворяют условию predicate. Примеры условий: [@class='some_сlass'] - отыскивает все узлы с классом some_class; [1] - первый узел.
* ...

<b>Примеры</b>
* /html/body/div/p - полный путь к узлу p с текстом "Какая-то умная надпись";
* //tbody//td - позволяет отыскать все элементы td, находящиеся в теле таблицы;
* //*[@id="note"] - позволяет отыскать элемент с id="note".

<b>Плюсы</b>
* Полноценный язык для работы со всеми xml документами, не только html.
* Позволяет перемещаться по дереву вверх и вниз.
* Есть поиск по тексту в узле и другие удобные вспомогательные функции.

<b>Минусы</b>
* Громоздкий.
* Сложный в освоении.

<b>Проверка через devtools<b>

В консоли браузера делаем
```javascript
document.evaluate("//div", document, null, XPathResult.FIRST_UNORDERED_NODE_ITERATOR_TYPE, null).iterateNext()
```
Документация по функции - https://developer.mozilla.org/en-US/docs/Web/API/Document/evaluate.

#### CSS селекторы
CSS (cascading style sheets) - каскадные таблицы стилей. Позволяют применить стиль к задаваемым CSS селекторами узлам. Эти же селекторы можно использовать для отыскания узлов в других задачах (например, парсинге). Применим только к html-документам.

<b>Синтаксис</b>

Можно посмотреть в https://scrapfly.io/blog/css-selector-cheatsheet/, https://www.w3schools.com/cssref/css_selectors.php.

* '#id' - позволяет получить узел с заданным идентификатором id;
* .class - позволяет получить узлы с классом class;
* [attribute="value"] - позволяет получать узлы со значением value атрибута attribute;
* '>' - позволяет получать дочерние узлы другого узла;
* '+' - позволяет получать узлы, следующие за узлом;
* ~ - позволяет получать узлы, предшествующие узлу.
* ...

<b>Примеры</b>

* '#note > p'. Получаем узел p, который является дочерним узлу с id=note.
* td.name. Получаем узлы td с классом name.
* tr[abc="def"]. Получаем узел tr со значением "def" аттрибута abc.

<b>Плюсы</b>

* Селекторы более лаконичны по сравнению с XPath.
* Проще в освоении.

<b>Минусы</b>

* Работает только с HTML страницами.
* Может переходить только от родительских узлов к дочерним, в обратную сторону нельзя.
* Меньше по сравнению с XPath встроенных возможностей.

<b>Проверка через devtools<b>

В консоли браузера делаем
```javascript
document.querySelectorAll('p.abc');
document.querySelector('p.abc');
```
Документация по функциям - https://developer.mozilla.org/en-US/docs/Web/API/Document/querySelector, https://developer.mozilla.org/en-US/docs/Web/API/Document/querySelectorAll.

### Использование из библиотек python

In [4]:
# css селекторы, библиотека beatiful soup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data, 'html.parser')

In [5]:
# один подходящий узел
node =  soup.select_one('#note > p')
node and node.text

'Какая-то умная надпись'

In [6]:
# все подходящие узлы
nodes =  soup.select('#note > p')
[*map(lambda x: x.text, nodes)]

['Какая-то умная надпись']

In [7]:
# XPath, библиотека lxml
import io
from lxml import etree
parser = etree.HTMLParser()
tree = etree.parse(io.StringIO(html_data), parser)
nodes = tree.xpath('//*[@id="note"]/p')
[*map(lambda x: x.text, nodes)]

['Какая-то умная надпись']

In [9]:
# Регулярные выражения
import re
re.search(r'8\(\d{3}\)\d{3}-\d{2}-\d{2}', html_data)

<re.Match object; span=(112, 127), match='8(906)123-45-32'>